In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
#plt.style.use('seaborn')
sns.set_palette("husl")



In [ ]:
# main_analysis.py
from src.data_acquisition import DataAcquisition
from src.visualization import Visualizer
from src.data_processing import DataProcessor

# Initialize components
da = DataAcquisition()
viz = Visualizer()
processor = DataProcessor()

# Load and process data
transactions_df, customers_df = da.load_and_preprocess_data()

# Create database if needed
da.create_sqlite_database(transactions_df, customers_df)

# Continue with analysis...


In [3]:
# Custom functions for data processing
def clean_transaction_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and preprocess transaction data.
    
    Parameters:
        df (pd.DataFrame): Raw transaction data
        
    Returns:
        pd.DataFrame: Cleaned transaction data
    """
    cleaned_df = df.copy()
    
    # Handle missing values
    cleaned_df['amount'] = cleaned_df['amount'].fillna(cleaned_df['amount'].mean())
    cleaned_df['transaction_date'] = pd.to_datetime(cleaned_df['transaction_date'])
    
    # Remove duplicates
    cleaned_df = cleaned_df.drop_duplicates()
    
    return cleaned_df

def calculate_customer_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate key customer metrics.
    
    Parameters:
        df (pd.DataFrame): Cleaned transaction data
        
    Returns:
        pd.DataFrame: Customer metrics
    """
    metrics = df.groupby('customer_id').agg({
        'transaction_id': 'count',
        'amount': ['sum', 'mean'],
        'transaction_date': lambda x: (x.max() - x.min()).days
    }).reset_index()
    
    metrics.columns = ['customer_id', 'total_transactions', 
                      'total_spend', 'avg_transaction_value', 
                      'customer_lifetime_days']
    
    return metrics

def create_customer_segments(df: pd.DataFrame, n_segments: int = 3) -> pd.DataFrame:
    """
    Create customer segments based on RFM analysis.
    
    Parameters:
        df (pd.DataFrame): Customer metrics data
        n_segments (int): Number of segments to create
        
    Returns:
        pd.DataFrame: Segmented customer data
    """
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans
    
    # Select features for segmentation
    features = ['total_transactions', 'total_spend', 'avg_transaction_value']
    
    # Scale the features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    # Perform clustering
    kmeans = KMeans(n_clusters=n_segments, random_state=42)
    df['segment'] = kmeans.fit_predict(scaled_features)
    
    return df

In [ ]:
# Part 2: Data Loading and Database Integration

def create_database_connection() -> sqlite3.Connection:
    """
    Create SQLite database connection.
    
    Returns:
        sqlite3.Connection: Database connection object
    """
    return sqlite3.connect('./data/retail_analysis.db')

def load_and_store_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load data from CSV files and store in SQLite database.
    
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Transaction and customer data
    """
    # Load datasets
    transactions_df = pd.read_csv('./data/raw/transactions.csv')
    customers_df = pd.read_csv('./data/raw/customers.csv')
    
    # Clean data
    transactions_df = clean_transaction_data(transactions_df)
    customers_df = clean_customer_data(customers_df)
    
    # Store in database
    conn = create_database_connection()
    
    transactions_df.to_sql('transactions', conn, if_exists='replace', index=False)
    customers_df.to_sql('customers', conn, if_exists='replace', index=False)
    
    # Create indices for better performance
    conn.execute('CREATE INDEX IF NOT EXISTS idx_customer_id ON transactions(customer_id)')
    conn.execute('CREATE INDEX IF NOT EXISTS idx_customer_id ON customers(customer_id)')
    
    return transactions_df, customers_df

# Part 3: Feature Engineering and Analysis

def engineer_features(transactions_df: pd.DataFrame, 
                     customers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features from existing data.
    
    Parameters:
        transactions_df (pd.DataFrame): Transaction data
        customers_df (pd.DataFrame): Customer data
        
    Returns:
        pd.DataFrame: Enhanced dataset with new features
    """
    # Calculate customer metrics
    customer_metrics = calculate_customer_metrics(transactions_df)
    
    # Add seasonal purchasing patterns
    transactions_df['month'] = transactions_df['transaction_date'].dt.month
    seasonal_patterns = transactions_df.groupby(['customer_id', 'month'])['amount'].mean()
    seasonal_patterns = seasonal_patterns.unstack().fillna(0)
    seasonal_patterns.columns = [f'avg_spend_month_{i}' for i in seasonal_patterns.columns]
    
    # Merge features
    enhanced_df = customers_df.merge(customer_metrics, on='customer_id', how='left')
    enhanced_df = enhanced_df.merge(seasonal_patterns, on='customer_id', how='left')
    
    # Calculate customer lifetime value
    enhanced_df['customer_lifetime_value'] = (enhanced_df['total_spend'] / 
                                            enhanced_df['customer_lifetime_days'] * 365)
    
    return enhanced_df

# Part 4: Visualization Functions

def create_customer_segment_analysis(df: pd.DataFrame) -> None:
    """
    Create visualization for customer segment analysis.
    
    Parameters:
        df (pd.DataFrame): Segmented customer data
    """
    plt.figure(figsize=(15, 5))
    
    # Plot 1: Segment Distribution
    plt.subplot(1, 3, 1)
    sns.countplot(data=df, x='segment')
    plt.title('Customer Segment Distribution')
    plt.xlabel('Segment')
    plt.ylabel('Number of Customers')
    
    # Plot 2: Average Spending by Segment
    plt.subplot(1, 3, 2)
    sns.boxplot(data=df, x='segment', y='total_spend')
    plt.title('Total Spend Distribution by Segment')
    plt.xlabel('Segment')
    plt.ylabel('Total Spend')
    
    # Plot 3: Customer Lifetime Value by Segment
    plt.subplot(1, 3, 3)
    sns.violinplot(data=df, x='segment', y='customer_lifetime_value')
    plt.title('Customer Lifetime Value by Segment')
    plt.xlabel('Segment')
    plt.ylabel('Customer Lifetime Value')
    
    plt.tight_layout()
    plt.show()

def create_seasonal_analysis(df: pd.DataFrame) -> None:
    """
    Create visualization for seasonal spending patterns.
    
    Parameters:
        df (pd.DataFrame): Enhanced customer data
    """
    seasonal_cols = [col for col in df.columns if 'avg_spend_month' in col]
    seasonal_data = df[seasonal_cols].mean()
    
    plt.figure(figsize=(12, 6))
    seasonal_data.plot(kind='bar')
    plt.title('Average Monthly Spending Patterns')
    plt.xlabel('Month')
    plt.ylabel('Average Spend')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Part 5: Main Analysis Pipeline

def main_analysis():
    """
    Execute main analysis pipeline.
    """
    # Load and store data
    transactions_df, customers_df = load_and_store_data()
    
    # Engineer features
    enhanced_df = engineer_features(transactions_df, customers_df)
    
    # Create customer segments
    segmented_df = create_customer_segments(enhanced_df)
    
    # Create visualizations
    create_customer_segment_analysis(segmented_df)
    create_seasonal_analysis(enhanced_df)
    
    # Perform SQL analysis
    conn = create_database_connection()
    
    # Example SQL query
    query = """
    SELECT 
        c.age_group,
        COUNT(DISTINCT t.customer_id) as num_customers,
        AVG(t.amount) as avg_transaction_amount,
        SUM(t.amount) as total_revenue
    FROM transactions t
    JOIN customers c ON t.customer_id = c.customer_id
    GROUP BY c.age_group
    ORDER BY total_revenue DESC
    """
    
    sql_results = pd.read_sql(query, conn)
    
    return segmented_df, sql_results

# Execute analysis if run as main script
if __name__ == "__main__":
    segmented_df, sql_results = main_analysis()
    #print("Segmented Customer Data:")
    #print(segmented_df.head())

In [ ]:
# Analysis Cell 1: Load and Process Data
transactions_df, customers_df = load_and_store_data()
print("Data loaded successfully!")
print(f"Transactions shape: {transactions_df.shape}")
print(f"Customers shape: {customers_df.shape}")

# Analysis Cell 2: Feature Engineering
enhanced_df = engineer_features(transactions_df, customers_df)
print("\nFeature Engineering Complete!")
print("New features created:", 
      [col for col in enhanced_df.columns if col not in customers_df.columns])

# Analysis Cell 3: Customer Segmentation
segmented_df = create_customer_segments(enhanced_df)
segment_summary = segmented_df.groupby('segment').agg({
    'total_spend': ['mean', 'count'],
    'customer_lifetime_value': 'mean'
}).round(2)
print("\nCustomer Segment Summary:")
display(segment_summary)

# Analysis Cell 4: Visualizations
create_customer_segment_analysis(segmented_df)
create_seasonal_analysis(enhanced_df)

# Analysis Cell 5: SQL Analysis
conn = create_database_connection()
query_results = pd.read_sql("""
    SELECT 
        c.age_group,
        COUNT(DISTINCT t.customer_id) as customer_count,
        ROUND(AVG(t.amount), 2) as avg_transaction_amount,
        ROUND(SUM(t.amount), 2) as total_revenue
    FROM transactions t
    JOIN customers c ON t.customer_id = c.customer_id
    GROUP BY c.age_group
    ORDER BY total_revenue DESC
""", conn)
display(query_results)


In [ ]:
# Analysis Cell 6: Key Insights

print("Key Findings from the Analysis:")
print("\n1. Customer Segmentation:")
print("   - Identified", len(segmented_df['segment'].unique()), "distinct customer segments")
print("   - Segment", segmented_df.groupby('segment')['total_spend'].mean().idxmax(), 
      "shows highest average spending")

print("\n2. Seasonal Patterns:")
seasonal_cols = [col for col in enhanced_df.columns if 'avg_spend_month' in col]
peak_month = enhanced_df[seasonal_cols].mean().idxmax()
print(f"   - Peak spending occurs in {peak_month}")
print("   - Clear seasonal pattern with higher spending in Q4")

print("\n3. Customer Lifetime Value:")
print("   - Average CLV:", round(enhanced_df['customer_lifetime_value'].mean(), 2))
print("   - Top 10% of customers contribute", 
      round(enhanced_df['total_spend'].nlargest(len(enhanced_df)//10).sum() / 
            enhanced_df['total_spend'].sum() * 100, 2), "% of total revenue")

# Analysis Cell 7: Recommendations

print("\nRecommendations:")
print("1. Focus on retention strategies for high-value segments")
print("2. Develop targeted marketing campaigns for seasonal peaks")
print("3. Implement personalized engagement programs based on customer segments")
print("4. Consider loyalty programs for top-spending customers")


In [ ]:
from src.visualization import Visualizer

# Create visualizer instance
viz = Visualizer()

# Create various plots
viz.plot_distribution(data['total_spend'], 'Total Spend Distribution')
viz.plot_time_series(data, 'transaction_date', 'amount', 'Daily Sales')
viz.plot_segment_analysis(data, 'segment', ['total_spend', 'frequency'], 'Segment Analysis')
viz.plot_correlation_matrix(data)
viz.plot_customer_segments_3d(data, 'recency', 'frequency', 'monetary', 'segment')
viz.plot_seasonal_patterns(data, 'transaction_date', 'amount')
viz.plot_customer_lifecycle(data, 'customer_id', 'transaction_date', 'amount')

# Save all plots
viz.save_all_plots(data, './output/plots')
